In [ ]:
from PLANBERT.Model import PLANBERT
# Load the training, validation, and testing set.
import pandas as pd
master_csv = pd.read_csv('./example/example_master.csv')
train_len, train_valid_len = 153681, 153681+31661
train_csv, valid_csv, test_csv = master_csv.iloc[:train_len], master_csv.iloc[train_len:train_valid_len], master_csv.iloc[train_valid_len:]

In [ ]:
# Train a PLAN-BERT with training set and validation set without checkpoint.
# [ Number of time slots, Number of items, [Number of features], ID of GPU]
planbert = PLANBERT(num_times=6, num_items=10000, feat_dims=[5000, 1000], cuda_num=0)
planbert.fit(train_csv, valid_csv, epoch_limit=2)
#planbert = PLANBERT(master_csv=train_csv, cuda_num=0)
#planbert.fit(train_csv, valid_csv)
#planbert.test(test_csv, h_list=[9], r_list=[3], pool_size=25)

In [ ]:
class recall_at_10(tf.keras.metrics.Metric):
    def __init__(self, name='recall_at_10', **kwargs):
        super(recall_at_10, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name='tp', initializer='zeros')

    def update_state(self, y_true, y_pred, use_pred):
        use_pred = use_pred[:, :, 0]
        # Tenth largest elements (output probs) over last axis 
        _, indices = tf.nn.top_k(y_pred, 10) # batch_size * num_semesters * 10
        # Multihot mask of top ten predictions (output classes) from y_pred
        # batch_size * num_semesters * 10 * num_courses
        y_pred_top = K.sum(K.one_hot(indices, K.int_shape(y_pred)[-1]), axis=-2)
        hit_mat = K.round(K.clip(y_true * y_pred_top, 0, 1))
        true_mat = K.round(K.clip(y_true, 0, 1))

        hit_sum = K.sum(K.sum(hit_mat, axis=-1) * use_pred)
        true_sum = K.sum(K.sum(true_mat, axis=-1) * use_pred)
        # recall_at_10 = 1 if there are no positive examples
        self.true_positives.assign_add(hit_sum / (true_sum + K.epsilon()))

    def result(self):
        return self.true_positives

    def reset_states(self):
        self.true_positives.assign(0)        

m = recall_at_10()

In [ ]:
m